### Step 3: Consolidate the Database

This step consolidates the `.xlsx` files generated in Step 2 into a unified database for analysis. The `all_text` column, created during this step, enhances the database for later semantic search operations by combining the `question` and `answers` columns into a single text field.

#### **Workflow**:
1. Process individual `.xlsx` files:
   - Standardize column names to `filename`, `question`, and `answers`.
   - Add an `Index` column for unique identification of rows.
   - Save the processed files in the `Database` folder.
2. Create a consolidated database:
   - Read all standardized `.xlsx` files from the `Database` folder.
   - Combine `question` and `answers` into a new `all_text` column for semantic searches.
   - Save the consolidated database as `ALL_PROCESSED.xlsx`.

#### **Inputs**:
- `.xlsx` files from the `Reviewed` folder.

#### **Outputs**:
- Standardized `.xlsx` files in the `Database` folder.
- A consolidated database file (`ALL_PROCESSED.xlsx`) with an `all_text` column for semantic analysis.


### Standardize Individual Files

The code block below processes all `.xlsx` files in the `Reviewed` folder:
1. Reads each file into a DataFrame.
2. Standardizes column names to `filename`, `question`, and `answers`.
3. Adds an `Index` column.
4. Saves the standardized version of the file to the `Database` folder.


In [ ]:
import os
import re
import csv
import pandas as pd
from docx import Document
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 0)  # Adapt the number as needed for your screen

# Directories
input_directory = "Reviewed"
output_directory = "Database"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Loop through all files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".xlsx"):
        filepath = os.path.join(input_directory, filename)
        
        # Read the Excel file
        df = pd.read_excel(filepath)

        # Ensure the first three columns have the names "country", "question", "answers"
        if len(df.columns) >= 3:
            df.columns.values[0:3] = ['filename', 'question', 'answers']
        
        # Add a new column with index numbers
        df['Index'] = range(1, len(df) + 1)
        
        # Save the modified DataFrame to the output directory
        output_filepath = os.path.join(output_directory, filename)
        df.to_excel(output_filepath, index=False)

        print(f"Processed and saved: {filename} to {output_filepath}")

### Consolidate All Files into a Unified Database

Next code block combines all standardized `.xlsx` files from the `Database` folder:
1. Reads each file into a DataFrame.
2. Ensures both `question` and `answers` columns exist; otherwise, creates an empty `all_text` column.
3. Concatenates `question` and `answers` into a new column `all_text`.
4. Appends all DataFrames into a single comprehensive DataFrame.
5. Saves the final database to `Database/ALL_PROCESSED.xlsx`.

##### **Purpose of `all_text`**:
The `all_text` column enables advanced semantic search functionality by merging `question` and `answers` into a single searchable field.

In [ ]:
# Append all country files into one data frame
directory = "Database"

# List to store DataFrames
dataframes = []

# Get all files in the directory and sort them alphabetically
files = sorted([f for f in os.listdir(directory) if f.endswith(".xlsx")])

# Loop through the sorted list of files
for filename in files:
    filepath = os.path.join(directory, filename)
    # Read each Excel file into a DataFrame
    df = pd.read_excel(filepath)
    
    # Check if both 'question' and 'answers' columns exist
    if 'question' in df.columns and 'answers' in df.columns:
        # Concatenate 'question' and 'answers' into a new column 'all_text'
        df['all_text'] = df['question'].astype(str) + " " + df['answers'].astype(str)
    else:
        # If either column is missing, add an empty 'all_text' column
        df['all_text'] = ""
    
    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames in the list
all_data = pd.concat(dataframes, ignore_index=True)

# Save the concatenated DataFrame to a new Excel file
all_data.to_excel("Database/ALL_PROCESSED.xlsx", index=False)

# View the structure
all_data

### Append Metadata Mapping

This section adds metadata information to the consolidated database using a separate metadata files (`Files_selected.xlsx`) and (`Files_selected.xlsx`)

#### Purpose:
Appending metadata information enriches the consolidated database by adding contextual details for each processed file. This is useful for downstream analysis and maintaining traceability of the data. Importantly, it adds infomration about the location of each file on ILO's internal drives.

#### Notes:
- The metadata file (`Files_selected.xlsx`) is available only to authorized ILO officials and is not included in the publicly available GitHub repository.
- Ensure the metadata file is correctly formatted and accessible in the working directory.

In [ ]:
# Append metadata from the main tracking file and additional regional labels for each country

# Read file with country labels from STATS
labels = pd.read_excel("/Data/LFS_labels.xlsx")

# Read metadata mapping of processed files
metadata_df = pd.read_excel("Files_selected.xlsx")

# Get the unique values of the 'filename' column from the 'all_data' dataframe
unique_files_processed = pd.DataFrame(all_data['filename'].unique(), columns=['filename'])
print(unique_files_processed)

# Merge the metadata into the full_data based on the 'filename' column
full_data = all_data.merge(metadata_df, on='filename', how='left')

# Rename the columns 
full_data = full_data.rename(columns={
    'filename': 'filename_processed',
    'Index': 'question_index',
    'value': 'questionnaire_filename',
    'path_Q': 'path',
    'ref_area': 'country',
    'time': 'year'
})

# Create a new column 'full_path' by concatenating 'path' and 'filename_questionnaire'
full_data['full_path'] = full_data['path'] + full_data['questionnaire_filename']

# Reorganize the columns in the desired order, drop those that are not of interest 
full_data = full_data[[
    'country', 'year', 'source', 'question_index', 'question', 'answers', 
    'questionnaire_filename', 'path', 'full_path', 'filename_processed', 'all_text'
]]

# Merge the labels into the full_data based on the 'question_index' or any other common column
full_data_with_labels = full_data.merge(labels, on='country', how='left')  # Adjust the 'on' column as needed

# Save the reorganized DataFrame to a new Excel file
full_data_with_labels.to_excel("Database/ALL_PROCESSED_METADATA.xlsx", index=False)

# Display the first 5 rows of the reorganized dataframe
full_data_with_labels.head(5)

In [ ]:
# Save key stats of the database into a separate file
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO

# 1. Number of distinct countries
distinct_countries = all_data['country'].nunique()

# 2. Number of rows per country
rows_per_country = all_data['country'].value_counts()

# 3. Total number of rows in the file
total_rows = len(all_data)

# Print the results
print(f"Number of distinct countries: {distinct_countries}")
print(f"Total number of rows: {total_rows}")
print("\nNumber of rows per country:")
print(rows_per_country)

# Create a DataFrame to store the overview results
stats_df = pd.DataFrame({
    'Statistic': ['Number of distinct countries', 'Total number of rows'],
    'Value': [distinct_countries, total_rows]
})

# Remove the '.xlsx' extension from the country names (if applicable)
rows_per_country.index = rows_per_country.index.str.replace('.xlsx', '', regex=False)

# Convert rows_per_country Series to DataFrame and reset index
rows_per_country_df = rows_per_country.reset_index()
rows_per_country_df.columns = ['Country', 'Number of Rows']

# Sort the DataFrame by the 'Country' column
rows_per_country_df = rows_per_country_df.sort_values(by='Country')

# Create a bar plot for the number of rows per country
plt.figure(figsize=(16, 12))
bars = plt.bar(rows_per_country.index, rows_per_country.values, color='skyblue')

# Add a title and labels
plt.title('Number of Questions per Country', fontsize=18)
plt.xlabel('Country', fontsize=14)
plt.ylabel('Number of Questions', fontsize=14)

# Rotate x-axis labels to 90 degrees
plt.xticks(rotation=90, fontsize=9)

# Add labels on top of each bar rotated at 90 degrees
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,  # X position
        height + max(rows_per_country.values) * 0.01,  # Slightly above the bar
        f'{int(height)}',  # Label text
        ha='center',
        va='bottom',
        rotation=90,
        fontsize=10
    )

# Adjust layout to fit all elements nicely
plt.tight_layout()

# Save the plot to a BytesIO object
plot_image = BytesIO()
plt.savefig(plot_image, format='png')
plt.close()  # Close the plot to free up memory
plot_image.seek(0)  # Go to the beginning of the BytesIO object

# Save all information into a single sheet in Main_stats.xlsx
with pd.ExcelWriter("Database_Main_stats.xlsx", engine='xlsxwriter') as writer:
    stats_df.to_excel(writer, sheet_name='Statistics', index=False, startrow=0)
    rows_per_country_df.to_excel(writer, sheet_name='Statistics', index=False, startrow=4)

    # Access the workbook and worksheet
    workbook = writer.book
    worksheet = writer.sheets['Statistics']

    # Set the width of the first column to be three times its default size
    worksheet.set_column('A:A', 30)  # Set column A (first column) to be 30 units wide
    worksheet.set_column('B:B', 20)  # Set column A (first column) to be 30 units wide
    # Insert the plot into the worksheet
    worksheet.insert_image('E2', 'Rows per Country Plot', {'image_data': plot_image})

### Conclusion

In this step, we:
1. Standardized individual `.xlsx` files generated in Step 2.
2. Merged all files into a single, unified database for analysis.
3. Created an automated overview of the database, with a countr of countries, Q-A pairs, years covered, etc.

### Next Steps
The consolidated database (`Database/ALL_PROCESSED_METADATA.xlsx`) is now ready for further analysis or integration in subsequent steps.